CIND 820- Big Data Analytics Project

Using Machine Learning for Prediction of Early Readmission of Diabetic Patients

Supervised by: Ceni Babaoglu

Presented by: Nehal Gamal Mohamed (501278190)


In [ ]:
!pip install pandas
import sys
!pip install matplotlib
!pip install graphviz

  Using cached pandas_profiling-3.6.6-py2.py3-none-any.whl (324 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.5/359.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/104.8 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 686.1/686.1 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 18.1 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27080 sha256=c15ab7f33daf86215c7d61789d63e83ee5ccd5d224511f5cfe4a04cfdf9981cb
  Stored in directory: /root/.cache/pip/wheels/dd/91/29/a79cecb328d01739e64017b6fb9a1ab9d8cb1853098ec5966d
Successfully built htmlmin
Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.

In [ ]:
#importing necessary libraries
import csv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


In [ ]:
#Uploading diabetic_data csv file
from google.colab import files
data = files.upload()
#Instintiating DataReader class
class DataReader:
    def read_csv(self, filename):
        df = pd.read_csv(filename)
        return df

filename = next(iter(data))
data_reader = DataReader()
df = data_reader.read_csv(filename)
#Displaying the first 10 records of the dataset
print(df.head(10))


Saving diabetic_data (version 1).csv to diabetic_data (version 1).csv
   encounter_id  patient_nbr             race  gender       age weight  \
0       2278392      8222157        Caucasian  Female    [0-10)      ?   
1        149190     55629189        Caucasian  Female   [10-20)      ?   
2         64410     86047875  AfricanAmerican  Female   [20-30)      ?   
3        500364     82442376        Caucasian    Male   [30-40)      ?   
4         16680     42519267        Caucasian    Male   [40-50)      ?   
5         35754     82637451        Caucasian    Male   [50-60)      ?   
6         55842     84259809        Caucasian    Male   [60-70)      ?   
7         63768    114882984        Caucasian    Male   [70-80)      ?   
8         12522     48330783        Caucasian  Female   [80-90)      ?   
9         15738     63555939        Caucasian  Female  [90-100)      ?   

   admission_type_id  discharge_disposition_id  admission_source_id  \
0                  6                        

In [ ]:
#Displaying variables data types
print("Dataset Information:")
print(df.info())

Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   encounter_id              101766 non-null  int64 
 1   patient_nbr               101766 non-null  int64 
 2   race                      101766 non-null  object
 3   gender                    101766 non-null  object
 4   age                       101766 non-null  object
 5   weight                    101766 non-null  object
 6   admission_type_id         101766 non-null  int64 
 7   discharge_disposition_id  101766 non-null  int64 
 8   admission_source_id       101766 non-null  int64 
 9   time_in_hospital          101766 non-null  int64 
 10  payer_code                101766 non-null  object
 11  medical_specialty         101766 non-null  object
 12  num_lab_procedures        101766 non-null  int64 
 13  num_procedures            101766 non-n

In [ ]:
#Defining lists for categorical variables and numeric variables
categorical_columns = [
    'race',
    'gender',
    'age',
    'weight',
    'payer_code',
    'medical_specialty',
    'diag_1',
    'diag_2',
    'diag_3',
    'max_glu_serum',
    'A1Cresult',
    'metformin',
    'repaglinide',
    'nateglinide',
    'chlorpropamide',
    'glimepiride',
    'acetohexamide',
    'glipizide',
    'glyburide',
    'tolbutamide',
    'pioglitazone',
    'rosiglitazone',
    'acarbose',
    'miglitol',
    'troglitazone',
    'tolazamide',
    'examide',
    'citoglipton',
    'insulin',
    'glyburide-metformin',
    'glipizide-metformin',
    'glimepiride-pioglitazone',
    'metformin-rosiglitazone',
    'metformin-pioglitazone',
    'change',
    'diabetesMed',
    'readmitted'
]

numeric_columns = [
    'encounter_id',
    'patient_nbr',
    'admission_type_id',
    'discharge_disposition_id',
    'admission_source_id',
    'time_in_hospital',
    'num_lab_procedures',
    'num_procedures',
    'num_medications',
    'number_outpatient',
    'number_emergency',
    'number_inpatient',
    'number_diagnoses'
]


In [ ]:
#Dataset Description
print("Dataset Description:")
print(df.describe(include='all'))

Dataset Description:
        encounter_id   patient_nbr       race  gender      age  weight  \
count   1.017660e+05  1.017660e+05     101766  101766   101766  101766   
unique           NaN           NaN          6       3       10      10   
top              NaN           NaN  Caucasian  Female  [70-80)       ?   
freq             NaN           NaN      76099   54708    26068   98569   
mean    1.652016e+08  5.433040e+07        NaN     NaN      NaN     NaN   
std     1.026403e+08  3.869636e+07        NaN     NaN      NaN     NaN   
min     1.252200e+04  1.350000e+02        NaN     NaN      NaN     NaN   
25%     8.496119e+07  2.341322e+07        NaN     NaN      NaN     NaN   
50%     1.523890e+08  4.550514e+07        NaN     NaN      NaN     NaN   
75%     2.302709e+08  8.754595e+07        NaN     NaN      NaN     NaN   
max     4.438672e+08  1.895026e+08        NaN     NaN      NaN     NaN   

        admission_type_id  discharge_disposition_id  admission_source_id  \
count       10

In [ ]:
#Converted blank and "?" to NaN to represent missing values
# Replace '?' and blank cells with NaN
df.replace({'?': pd.NA, '': pd.NA}, inplace=True)

# Replaced 'None' in 'max_glu_serum' and 'A1Cresult' to be seen as 'No Test' category and not as missing values
df['max_glu_serum'].replace({pd.NA: 'No Test', 'None': 'No Test'}, inplace=True)
df['A1Cresult'].replace({pd.NA: 'No Test', 'None': 'No Test'}, inplace=True)
missing_values = df.isna().sum()
print("Number of missing values in each column:")
print(missing_values)

Number of missing values in each column:
encounter_id                    0
patient_nbr                     0
race                         2273
gender                          0
age                             0
weight                      98569
admission_type_id               0
discharge_disposition_id        0
admission_source_id             0
time_in_hospital                0
payer_code                  40256
medical_specialty           49949
num_lab_procedures              0
num_procedures                  0
num_medications                 0
number_outpatient               0
number_emergency                0
number_inpatient                0
diag_1                         21
diag_2                        358
diag_3                       1423
number_diagnoses                0
max_glu_serum                   0
A1Cresult                       0
metformin                       0
repaglinide                     0
nateglinide                     0
chlorpropamide                  0
glimepi

In [ ]:
#Handling missing values in categorical columns by adding "Missing" as subcategory
for col in categorical_columns:
    df[col].fillna('Missing', inplace=True)
print(df[categorical_columns].isnull().sum())



race                        0
gender                      0
age                         0
weight                      0
payer_code                  0
medical_specialty           0
diag_1                      0
diag_2                      0
diag_3                      0
max_glu_serum               0
A1Cresult                   0
metformin                   0
repaglinide                 0
nateglinide                 0
chlorpropamide              0
glimepiride                 0
acetohexamide               0
glipizide                   0
glyburide                   0
tolbutamide                 0
pioglitazone                0
rosiglitazone               0
acarbose                    0
miglitol                    0
troglitazone                0
tolazamide                  0
examide                     0
citoglipton                 0
insulin                     0
glyburide-metformin         0
glipizide-metformin         0
glimepiride-pioglitazone    0
metformin-rosiglitazone     0
metformin-

In [ ]:
# converting numeric columns to numeric data types and categorical columns to category data type
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

for col in categorical_columns:
    if col in df.columns:
        df[col] = df[col].astype('category')
print(df.dtypes)
print(df[numeric_columns].dtypes)


encounter_id                   int64
patient_nbr                    int64
race                        category
gender                      category
age                         category
weight                      category
admission_type_id              int64
discharge_disposition_id       int64
admission_source_id            int64
time_in_hospital               int64
payer_code                  category
medical_specialty           category
num_lab_procedures             int64
num_procedures                 int64
num_medications                int64
number_outpatient              int64
number_emergency               int64
number_inpatient               int64
diag_1                      category
diag_2                      category
diag_3                      category
number_diagnoses               int64
max_glu_serum               category
A1Cresult                   category
metformin                   category
repaglinide                 category
nateglinide                 category
c

In [ ]:
# Check for missing values
missing_values = df.isnull().sum()
print(missing_values)

encounter_id                0
patient_nbr                 0
race                        0
gender                      0
age                         0
weight                      0
admission_type_id           0
discharge_disposition_id    0
admission_source_id         0
time_in_hospital            0
payer_code                  0
medical_specialty           0
num_lab_procedures          0
num_procedures              0
num_medications             0
number_outpatient           0
number_emergency            0
number_inpatient            0
diag_1                      0
diag_2                      0
diag_3                      0
number_diagnoses            0
max_glu_serum               0
A1Cresult                   0
metformin                   0
repaglinide                 0
nateglinide                 0
chlorpropamide              0
glimepiride                 0
acetohexamide               0
glipizide                   0
glyburide                   0
tolbutamide                 0
pioglitazo

In [ ]:
#Checking for duplicate records
duplicate_records = df.duplicated().sum()
print(f"Number of duplicate records: {duplicate_records}")

Number of duplicate records: 0


In [ ]:
# Here I wanted to clean the dataframe by removing irrelevant columns, columns with high missing values
df_cleaned = df.copy()
columns_to_drop = ['encounter_id', 'patient_nbr', 'weight', 'payer_code']
df_cleaned.drop(columns=columns_to_drop, inplace=True)

# Here I put a threshold of 95% for List of columns dominated by "No" category since they have very low variance and no added value
columns_dominated_by_no = [
    'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
    'glimepiride', 'acetohexamide', 'glipizide', 'glyburide',
    'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose',
    'troglitazone', 'tolazamide', 'examide', 'citoglipton',
    'glyburide-metformin', 'glipizide-metformin', 'glimepiride-pioglitazone',
    'metformin-rosiglitazone', 'metformin-pioglitazone'
]

threshold = 0.95
columns_to_drop_dominated = []
for col in columns_dominated_by_no:
    if df_cleaned[col].value_counts(normalize=True).get('No', 0) > threshold:
        columns_to_drop_dominated.append(col)
df_cleaned.drop(columns=columns_to_drop_dominated, inplace=True)

# Check the remaining columns
print("Remaining columns after dropping unnecessary ones:")
print(df_cleaned.columns)
print(df_cleaned.info())

Remaining columns after dropping unnecessary ones:
Index(['race', 'gender', 'age', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'medical_specialty', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'diag_1', 'diag_2', 'diag_3', 'number_diagnoses',
       'max_glu_serum', 'A1Cresult', 'metformin', 'glimepiride', 'glipizide',
       'glyburide', 'pioglitazone', 'rosiglitazone', 'miglitol', 'insulin',
       'change', 'diabetesMed', 'readmitted'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 31 columns):
 #   Column                    Non-Null Count   Dtype   
---  ------                    --------------   -----   
 0   race                      101766 non-null  category
 1   gender                    101766 non-null  category
 2   age                       101766 non-nu

In [ ]:
# Identify and verifying remaining categorical columns for onehot coding
categorical_columns = df_cleaned.select_dtypes(include=['category']).columns.tolist()
print("Categorical columns for one-hot encoding:", categorical_columns)


Categorical columns for one-hot encoding: ['race', 'gender', 'age', 'medical_specialty', 'diag_1', 'diag_2', 'diag_3', 'max_glu_serum', 'A1Cresult', 'metformin', 'glimepiride', 'glipizide', 'glyburide', 'pioglitazone', 'rosiglitazone', 'miglitol', 'insulin', 'change', 'diabetesMed', 'readmitted']


In [ ]:
# One-hot encode the remaining categorical columns
df_onehot_coded = pd.get_dummies(df_cleaned, columns=categorical_columns, drop_first=False)
boolean_columns = df_onehot_coded.select_dtypes(include='bool').columns
df_onehot_coded[boolean_columns] = df_onehot_coded[boolean_columns].astype(int)
print(df_onehot_coded.head())
print(df_onehot_coded.columns)


   admission_type_id  discharge_disposition_id  admission_source_id  \
0                  6                        25                    1   
1                  1                         1                    7   
2                  1                         1                    7   
3                  1                         1                    7   
4                  1                         1                    7   

   time_in_hospital  num_lab_procedures  num_procedures  num_medications  \
0                 1                  41               0                1   
1                 3                  59               0               18   
2                 2                  11               5               13   
3                 2                  44               1               16   
4                 1                  51               0                8   

   number_outpatient  number_emergency  number_inpatient  ...  insulin_No  \
0                  0                 0 

In [ ]:
# To assess the normality of the numerical variables, I conducted the Shapiro-Wilk test.
# Despite the test results indicating that none of the variables followed a normal distribution,
# I chose to retain the outliers. The reason behind this decision was that the count of outliers
# across the numeric variables was not substantial, and removing them could potentially reduce
# the variance and valuable information in the dataset.
from scipy.stats import shapiro
dependent_var_columns = ['readmitted_>30', 'readmitted_NO', 'readmitted_<30']
numerical_columns = df_onehot_coded.select_dtypes(include=[np.number]).columns
numerical_columns = numerical_columns.drop(dependent_var_columns)

# Perform the Shapiro-Wilk test
shapiro_results = {}
for col in numerical_columns:
    stat, p_value = shapiro(df_onehot_coded[col])
    shapiro_results[col] = (stat, p_value)
shapiro_df = pd.DataFrame(shapiro_results, index=['Statistic', 'p-value']).T
display(shapiro_df)


/usr/local/lib/python3.10/dist-packages/scipy/stats/_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


,Statistic,p-value
admission_type_id,0.721419,0.0
discharge_disposition_id,0.568973,0.0
admission_source_id,0.737137,0.0
time_in_hospital,0.886919,0.0
num_lab_procedures,0.984858,0.0
...,...,...
insulin_Up,0.363303,0.0
change_Ch,0.634863,0.0
change_No,0.635076,0.0
diabetesMed_No,0.520568,0.0


In [ ]:
#Next step is to normalize the independent variables to insure scalability
# Function to normalize numerical columns
def normalize(x):
    return (x - x.min()) / (x.max() - x.min())
dependent_var_columns = ['readmitted_>30', 'readmitted_NO', 'readmitted_<30']
numerical_columns = df_onehot_coded.select_dtypes(include=[np.number]).columns
numerical_columns = numerical_columns.drop(dependent_var_columns)
for col in numerical_columns:
    df_onehot_coded[col] = normalize(df_onehot_coded[col])
print(df_onehot_coded.head())



   admission_type_id  discharge_disposition_id  admission_source_id  \
0           0.714286                  0.888889                 0.00   
1           0.000000                  0.000000                 0.25   
2           0.000000                  0.000000                 0.25   
3           0.000000                  0.000000                 0.25   
4           0.000000                  0.000000                 0.25   

   time_in_hospital  num_lab_procedures  num_procedures  num_medications  \
0          0.000000            0.305344        0.000000           0.0000   
1          0.153846            0.442748        0.000000           0.2125   
2          0.076923            0.076336        0.833333           0.1500   
3          0.076923            0.328244        0.166667           0.1875   
4          0.000000            0.381679        0.000000           0.0875   

   number_outpatient  number_emergency  number_inpatient  ...  insulin_No  \
0           0.000000               0.0 

In [ ]:
# Checking if the numerical columns are normalized
for col in numerical_columns:
    min_val = df_onehot_coded[col].min()
    max_val = df_onehot_coded[col].max()
    print(f'{col}: min = {min_val}, max = {max_val}')


admission_type_id: min = 0.0, max = 1.0
discharge_disposition_id: min = 0.0, max = 1.0
admission_source_id: min = 0.0, max = 1.0
time_in_hospital: min = 0.0, max = 1.0
num_lab_procedures: min = 0.0, max = 1.0
num_procedures: min = 0.0, max = 1.0
num_medications: min = 0.0, max = 1.0
number_outpatient: min = 0.0, max = 1.0
number_emergency: min = 0.0, max = 1.0
number_inpatient: min = 0.0, max = 1.0
number_diagnoses: min = 0.0, max = 1.0
race_AfricanAmerican: min = 0.0, max = 1.0
race_Asian: min = 0.0, max = 1.0
race_Caucasian: min = 0.0, max = 1.0
race_Hispanic: min = 0.0, max = 1.0
race_Missing: min = 0.0, max = 1.0
race_Other: min = 0.0, max = 1.0
gender_Female: min = 0.0, max = 1.0
gender_Male: min = 0.0, max = 1.0
gender_Unknown/Invalid: min = 0.0, max = 1.0
age_[0-10): min = 0.0, max = 1.0
age_[10-20): min = 0.0, max = 1.0
age_[20-30): min = 0.0, max = 1.0
age_[30-40): min = 0.0, max = 1.0
age_[40-50): min = 0.0, max = 1.0
age_[50-60): min = 0.0, max = 1.0
age_[60-70): min = 0.0, 

In [ ]:
# Here is the Classification starts by setting the dependent variable and splitting the dataset to train and test set
from sklearn.model_selection import train_test_split
X = df_onehot_coded.drop(columns=['readmitted_>30', 'readmitted_NO', 'readmitted_<30'])
y = df_onehot_coded['readmitted_<30']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# The first ML Model ogistic regression model
log_reg = LogisticRegression(max_iter=1000,random_state=42)
log_reg.fit(X_train, y_train)
y_pred_lr = log_reg.predict(X_test)

# Evaluate the model
print("Logistic Regression Classifier")
print(classification_report(y_test, y_pred_lr))
print(confusion_matrix(y_test, y_pred_lr))



Logistic Regression Classifier
              precision    recall  f1-score   support

           0       0.89      1.00      0.94     18069
           1       0.43      0.02      0.03      2285

    accuracy                           0.89     20354
   macro avg       0.66      0.51      0.49     20354
weighted avg       0.84      0.89      0.84     20354

[[18021    48]
 [ 2249    36]]


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix

# The second ML algorithm Decision Tree
decision_tree = DecisionTreeClassifier(random_state=42)
decision_tree.fit(X_train, y_train)
y_pred_tree = decision_tree.predict(X_test)

# Evaluate the model
print("Decision Tree Classifier")
print(classification_report(y_test, y_pred_tree))
print(confusion_matrix(y_test, y_pred_tree))



Decision Tree Classifier
              precision    recall  f1-score   support

           0       0.89      0.91      0.90     18069
           1       0.17      0.16      0.16      2285

    accuracy                           0.82     20354
   macro avg       0.53      0.53      0.53     20354
weighted avg       0.81      0.82      0.82     20354

[[16383  1686]
 [ 1930   355]]


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
# The Third ML algorithm Random Forest
# Here I used the class weight attribute in the Random Forest classifier(The bagging technique) it help with the class label imbalance
# Initialize the Random Forest model with class_weight='balanced_subsample'
random_forest = RandomForestClassifier(class_weight='balanced_subsample', random_state=42)
random_forest.fit(X_train, y_train)
y_pred_rf = random_forest.predict(X_test)

# Evaluate the model
accuracy_rf = accuracy_score(y_test, y_pred_rf)
conf_matrix_rf = confusion_matrix(y_test, y_pred_rf)
class_report_rf = classification_report(y_test, y_pred_rf)

print(f'Random Forest Accuracy: {accuracy_rf}')
print('Random Forest Confusion Matrix:')
print(conf_matrix_rf)
print('Random Forest Classification Report:')
print(class_report_rf)



Random Forest Accuracy: 0.8880809668861157
Random Forest Confusion Matrix:
[[18066     3]
 [ 2275    10]]
Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.89      1.00      0.94     18069
           1       0.77      0.00      0.01      2285

    accuracy                           0.89     20354
   macro avg       0.83      0.50      0.47     20354
weighted avg       0.87      0.89      0.84     20354



In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# The forth ML algorithm Boosting Classifier
gradient_boosting = GradientBoostingClassifier(random_state=42)
gradient_boosting.fit(X_train, y_train)
y_pred_gb = gradient_boosting.predict(X_test)

# Evaluate the model
accuracy_gb = accuracy_score(y_test, y_pred_gb)
conf_matrix_gb = confusion_matrix(y_test, y_pred_gb)
class_report_gb = classification_report(y_test, y_pred_gb)

print(f'Gradient Boosting Accuracy: {accuracy_gb}')
print('Gradient Boosting Confusion Matrix:')
print(conf_matrix_gb)
print('Gradient Boosting Classification Report:')
print(class_report_gb)



Gradient Boosting Accuracy: 0.8881300972781763
Gradient Boosting Confusion Matrix:
[[18051    18]
 [ 2259    26]]
Gradient Boosting Classification Report:
              precision    recall  f1-score   support

           0       0.89      1.00      0.94     18069
           1       0.59      0.01      0.02      2285

    accuracy                           0.89     20354
   macro avg       0.74      0.51      0.48     20354
weighted avg       0.86      0.89      0.84     20354



In [ ]:
from imblearn.over_sampling import SMOTE
from collections import Counter

# As we can see from initial results of the previous 4 ML algorithms that the class imbalance is affecting on the performance
# specially on the RECALL which represents the the people who were readmitted within a month time
# and were wrongly predicted as readmitted after a month time
# So, we need to handle the class imbalance by more representing the minority class using SMOTE
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Check the class distribution after resampling
print("Resampled training set class distribution:", Counter(y_train_smote))



In [ ]:
# Random undersampling
# Here I tried to handle the class imbalance by more representing the minority class and less representation of the majority class
# using Random undersampling
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
X = df_onehot_coded.drop(columns=['readmitted_>30', 'readmitted_NO', 'readmitted_<30'])
y = df_onehot_coded['readmitted_<30']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply Random Under-Sampling to the training data
rus = RandomUnderSampler(random_state=42)
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

# Logistic Regression with RUS
log_reg_rus = LogisticRegression(max_iter=1000, random_state=42)
log_reg_rus.fit(X_train_rus, y_train_rus)

# Evaluate the model
y_pred_lr = log_reg_rus.predict(X_test)
accuracy_lr = accuracy_score(y_test, y_pred_lr)
conf_matrix_lr = confusion_matrix(y_test, y_pred_lr)
class_report_lr = classification_report(y_test, y_pred_lr)

print('Logistic Regression with RUS')
print(f'Accuracy: {accuracy_lr}')
print('Confusion Matrix:')
print(conf_matrix_lr)
print('Classification Report:')
print(class_report_lr)


Logistic Regression with RUS
Accuracy: 0.6109364252726737
Confusion Matrix:
[[11141  6928]
 [  991  1294]]
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.62      0.74     18069
           1       0.16      0.57      0.25      2285

    accuracy                           0.61     20354
   macro avg       0.54      0.59      0.49     20354
weighted avg       0.83      0.61      0.68     20354



In [ ]:
from sklearn.tree import DecisionTreeClassifier
# Decision Tree with RUS
decision_tree_rus = DecisionTreeClassifier(random_state=42)
decision_tree_rus.fit(X_train_rus, y_train_rus)

# Evaluate the model
y_pred_dt = decision_tree_rus.predict(X_test)
accuracy_dt = accuracy_score(y_test, y_pred_dt)
conf_matrix_dt = confusion_matrix(y_test, y_pred_dt)
class_report_dt = classification_report(y_test, y_pred_dt)

print('Decision Tree with RUS')
print(f'Accuracy: {accuracy_dt}')
print('Confusion Matrix:')
print(conf_matrix_dt)
print('Classification Report:')
print(class_report_dt)


Decision Tree with RUS
Accuracy: 0.5648029871278373
Confusion Matrix:
[[10218  7851]
 [ 1007  1278]]
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.57      0.70     18069
           1       0.14      0.56      0.22      2285

    accuracy                           0.56     20354
   macro avg       0.53      0.56      0.46     20354
weighted avg       0.82      0.56      0.64     20354



In [ ]:
from sklearn.ensemble import RandomForestClassifier
# Random Forest with RUS
random_forest_rus = RandomForestClassifier(class_weight='balanced_subsample', random_state=42)
random_forest_rus.fit(X_train_rus, y_train_rus)

# Evaluate the model
y_pred_rf = random_forest_rus.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
conf_matrix_rf = confusion_matrix(y_test, y_pred_rf)
class_report_rf = classification_report(y_test, y_pred_rf)

print('Random Forest with RUS')
print(f'Accuracy: {accuracy_rf}')
print('Confusion Matrix:')
print(conf_matrix_rf)
print('Classification Report:')
print(class_report_rf)


Random Forest with RUS
Accuracy: 0.6215485899577479
Confusion Matrix:
[[11270  6799]
 [  904  1381]]
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.62      0.75     18069
           1       0.17      0.60      0.26      2285

    accuracy                           0.62     20354
   macro avg       0.55      0.61      0.50     20354
weighted avg       0.84      0.62      0.69     20354



In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
# Gradient Boosting with RUS
gradient_boosting_rus = GradientBoostingClassifier(random_state=42)
gradient_boosting_rus.fit(X_train_rus, y_train_rus)

# Evaluate the model
y_pred_gb = gradient_boosting_rus.predict(X_test)
accuracy_gb = accuracy_score(y_test, y_pred_gb)
conf_matrix_gb = confusion_matrix(y_test, y_pred_gb)
class_report_gb = classification_report(y_test, y_pred_gb)

print('Gradient Boosting with RUS')
print(f'Accuracy: {accuracy_gb}')
print('Confusion Matrix:')
print(conf_matrix_gb)
print('Classification Report:')
print(class_report_gb)


Gradient Boosting with RUS
Accuracy: 0.6251351085781665
Confusion Matrix:
[[11281  6788]
 [  842  1443]]
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.62      0.75     18069
           1       0.18      0.63      0.27      2285

    accuracy                           0.63     20354
   macro avg       0.55      0.63      0.51     20354
weighted avg       0.85      0.63      0.69     20354

